<a href="https://colab.research.google.com/github/lweislo/CNResults/blob/master/tissot_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tissot scraping test
* Press play to load


In [50]:
import requests
import html5lib
import pandas as pd
from bs4 import BeautifulSoup
import os
import json
import numpy as np
import datetime
from google.colab import files
import re
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

race_bibs = {1:"Mathias Frank (Swi) AG2R la Mondiale",
2:"Geoffrey Bouchard (Fra) AG2R la Mondiale",
3:"Silvan Dillier (Swi) AG2R la Mondiale",
4:"Axel Domont (Fra) AG2R la Mondiale",
5:"Jaakko Hanninen (Fin) AG2R la Mondiale",
6:"Andrea Vendrame (Ita) AG2R la Mondiale",
7:"Lawrence Warbasse (USA) AG2R la Mondiale",
11:"Mathieu Van der Poel (Ned) Alpecin-Fenix",
12:"Tim Merlier (Bel) Alpecin-Fenix",
13:"Jonas Rickaert (Bel) Alpecin-Fenix",
14:"Kristian Sbaragli (Ita) Alpecin-Fenix",
15:"Dries De Bondt (Bel) Alpecin-Fenix",
16:"Otto Vergaerde (Bel) Alpecin-Fenix",
17:"Louis Vervaeke (Bel) Alpecin-Fenix",
21:"Nicola Bagioli (Ita) Androni Giocattoli-Sidermec",
22:"Davide Gabburo (Ita) Androni Giocattoli-Sidermec",
23:"Luca Pacioni (Ita) Androni Giocattoli-Sidermec",
24:"Simon Pellaud (Swi) Androni Giocattoli-Sidermec",
25:"Jhonatan Restrepo Valencia (Col) Androni Giocattoli-Sidermec",
26:"Kevin Rivera Serrano (CRc) Androni Giocattoli-Sidermec",
27:"Josip Rumac (Cro) Androni Giocattoli-Sidermec",
31:"Jakob Fuglsang (Den) Astana Pro Team",
32:"Alex Aranburu Deba (Spa) Astana Pro Team",
33:"Manuele Boaro (Ita) Astana Pro Team",
34:"Fabio Felline (Ita) Astana Pro Team",
35:"Yuriy Natarov (Kaz) Astana Pro Team",
36:"Oscar Rodriguez Garaicoechea (Spa) Astana Pro Team",
37:"Aleksandr Vlasov (Rus) Astana Pro Team",
41:"Dylan Teuns (Bel) Bahrain McLaren",
42:"Phil Bauhaus (Ger) Bahrain McLaren",
43:"Ivan Garcia Cortina (Spa) Bahrain McLaren",
44:"Heinrich Haussler (Aus) Bahrain McLaren",
45:"Mark Padun (Ukr) Bahrain McLaren",
46:"Hermann Pernsteiner (Aut) Bahrain McLaren",
47:"Jan Tratnik (Slo) Bahrain McLaren",
51:"Giovanni Carboni (Ita) Bardiani CSF Faizane'",
52:"Giovanni Lonardi (Ita) Bardiani CSF Faizane'",
53:"Fabio Mazzucco (Ita) Bardiani CSF Faizane'",
54:"Umberto Orsini (Ita) Bardiani CSF Faizane'",
55:"Francesco Romano (Ita) Bardiani CSF Faizane'",
56:"Daniel Savini (Ita) Bardiani CSF Faizane'",
57:"Alessandro Tonelli (Ita) Bardiani CSF Faizane'",
61:"Rafal Majka (Pol) Bora-Hansgrohe",
62:"Pascal Ackermann (Ger) Bora-Hansgrohe",
63:"Maciej Bodnar (Pol) Bora-Hansgrohe",
64:"Matteo Fabbro (Ita) Bora-Hansgrohe",
65:"Patrick Konrad (Aut) Bora-Hansgrohe",
66:"Pawel Poljanski (Pol) Bora-Hansgrohe",
67:"Rüdiger Selig (Ger) Bora-Hansgrohe",
71:"William Barta (USA) CCC Team",
72:"Pavel Kochetkov (Rus) CCC Team",
73:"Victor De la Parte (Spa) CCC Team",
74:"Lukasz Wisniowski (Pol) CCC Team",
75:"Szymon Sajnok (Pol) CCC Team",
76:"Nathan Van Hooydonck (Bel) CCC Team",
77:"Georg Zimmermann (Ger) CCC Team",
81:"Piet Allegaert (Bel) Cofidis",
82:"Dimitri Claeys (Bel) Cofidis",
83:"Nathan Haas (Aus) Cofidis",
84:"Fabio Sabatini (Ita) Cofidis",
85:"Kenneth Vanbilsen (Bel) Cofidis",
86:"Julien Vermote (Bel) Cofidis",
87:"Attilio Viviani (Ita) Cofidis",
91:"Davide Ballerini (Ita) Deceuninck-Quickstep",
92:"Stijn Steels (Bel) Deceuninck-Quickstep",
93:"Iljo Keisse (Bel) Deceuninck-Quickstep",
94:"James Knox (GBr) Deceuninck-Quickstep",
95:"Fausto Masnada (Ita) Deceuninck-Quickstep",
96:"Florian Senechal (Fra) Deceuninck-Quickstep",
97:"Bert Van Lerberghe (Bel) Deceuninck-Quickstep",
101:"Michael Woods (Can) EF Pro Cycling",
102:"Jonathan Caicedo (Ecu) EF Pro Cycling",
103:"Simon Clarke (Aus) EF Pro Cycling",
104:"Lawson Craddock (USA) EF Pro Cycling",
105:"Ruben Guerreiro (Por) EF Pro Cycling",
106:"Magnus Cort Nielsen (Den) EF Pro Cycling",
107:"Tanel Kangert (Est) EF Pro Cycling",
111:"Igor Boev (Rus) Gazprom-Rusvelo",
112:"Marco Canola (Ita) Gazprom-Rusvelo",
113:"Imerio Cima (Ita) Gazprom-Rusvelo",
114:"Denis Nekrasov (Rus) Gazprom-Rusvelo",
115:"Ivan Rovny (Rus) Gazprom-Rusvelo",
116:"Aleksei Rybalkin (Rus) Gazprom-Rusvelo",
117:"Simone Velasco (Ita) Gazprom-Rusvelo",
121:"Anthony Roux (Fra) Groupama-FDJ",
122:"Kilian Frankiny (Swi) Groupama-FDJ",
123:"Olivier Le Gac (Fra) Groupama-FDJ",
124:"Bruno Armirail (Fra) Groupama-FDJ",
125:"Romain Seigle (Fra) Groupama-FDJ",
126:"Benjamin Thomas (Fra) Groupama-FDJ",
127:"Tobias Ludvigsson (Swe) Groupama-FDJ",
131:"Matthias Brändle (Aut) Israel Start-Up Nation",
132:"Alexander Cataford (Can) Israel Start-Up Nation",
133:"Davide Cimolai (Ita) Israel Start-Up Nation",
134:"Alex Dowsett (GBr) Israel Start-Up Nation",
135:"Daniel Navarro Garcia (Spa) Israel Start-Up Nation",
136:"Guy Sagiv (Isr) Israel Start-Up Nation",
137:"Rick Zabel (Ger) Israel Start-Up Nation",
141:"Adam Hansen (Aus) Lotto Soudal",
142:"Kobe Goossens (Bel) Lotto Soudal",
143:"Carl Fredrik Hagen (Nor) Lotto Soudal",
144:"Matthew Holmes (GBr) Lotto Soudal",
145:"Nikolas Maes (Bel) Lotto Soudal",
146:"Stefano Oldani (Ita) Lotto Soudal",
147:"Florian Vermeersch (Bel) Lotto Soudal",
151:"Simon Yates (GBr) Mitchelton-Scott",
152:"Edoardo Affini (Ita) Mitchelton-Scott",
153:"Brent Bookwalter (USA) Mitchelton-Scott",
154:"Jack Haig (Aus) Mitchelton-Scott",
155:"Lucas Hamilton (Aus) Mitchelton-Scott",
156:"Michael Hepburn (Aus) Mitchelton-Scott",
157:"Cameron Meyer (Aus) Mitchelton-Scott",
161:"Davide Villella (Ita) Movistar Team",
162:"Juan Diego Alba Bolivar (Col) Movistar Team",
163:"Hector Carretero (Spa) Movistar Team",
164:"Matteo Jorgenson (USA) Movistar Team",
165:"Sergio Samitier Samitier (Spa) Movistar Team",
166:"Eduardo Sepulveda (Arg) Movistar Team",
167:"Albert Torres Barcelo (Spa) Movistar Team",
171:"Louis Meintjes (RSA) NTT Pro Cycling",
172:"Carlos Barbero (Spa) NTT Pro Cycling",
173:"Samuele Battistella (Ita) NTT Pro Cycling",
174:"Victor Campenaerts (Bel) NTT Pro Cycling",
175:"Benjamin Dyball (Aus) NTT Pro Cycling",
176:"Enrico Gasparotto (Swi) NTT Pro Cycling",
177:"Amanuel Gebreigzabhier (Eri) NTT Pro Cycling",
181:"Geraint Thomas (GBr) Ineos Grenadiers",
182:"Rohan Dennis (Aus) Ineos Grenadiers",
183:"Edward Dunbar (Irl) Ineos Grenadiers",
184:"Chris Froome (GBr) Ineos Grenadiers",
185:"Filippo Ganna (Ita) Ineos Grenadiers",
186:"Tao Geoghegan Hart (GBr) Ineos Grenadiers",
187:"Salvatore Puccio (Ita) Ineos Grenadiers",
191:"Pascal Eenkhoorn (Ned) Team Jumbo-Visma",
192:"Paul Martens (Ger) Team Jumbo-Visma",
193:"Christoph Pfingsten (Ger) Team Jumbo-Visma",
194:"Timo Roosen (Ned) Team Jumbo-Visma",
195:"Mike Teunissen (Ned) Team Jumbo-Visma",
196:"Jos Van Emden (Ned) Team Jumbo-Visma",
197:"Maarten Wynants (Bel) Team Jumbo-Visma",
201:"Michael Matthews (Aus) Team Sunweb",
202:"Alberto Dainese (Ita) Team Sunweb",
203:"Christopher Hamilton (Aus) Team Sunweb",
204:"Jai Hindley (Aus) Team Sunweb",
205:"Wilco Kelderman (Ned) Team Sunweb",
206:"Sam Oomen (Ned) Team Sunweb",
207:"Martijn Tusveld (Ned) Team Sunweb",
211:"Niki Terpstra (Ned) Total Direct Energie",
212:"Jonathan Hivert (Fra) Total Direct Energie",
213:"Lorrenzo Manzin (Fra) Total Direct Energie",
214:"Adrien Petit (Fra) Total Direct Energie",
215:"Julien Simon (Fra) Total Direct Energie",
216:"Rein Taaramäe (Est) Total Direct Energie",
217:"Dries Van Gestel (Bel) Total Direct Energie",
221:"Vincenzo Nibali (Ita) Trek-Segafredo",
222:"Julien Bernard (Fra) Trek-Segafredo",
223:"Gianluca Brambilla (Ita) Trek-Segafredo",
224:"Alex Kirsch (Lux) Trek-Segafredo",
225:"Koen De Kort (Ned) Trek-Segafredo",
226:"Antonio Nibali (Ita) Trek-Segafredo",
227:"Pieter Weening (Ned) Trek-Segafredo",
231:"Fernando Gaviria Rendon (Col) UAE Team Emirates",
232:"Mikkel Bjerg (Den) UAE Team Emirates",
233:"Rui Costa (Por) UAE Team Emirates",
234:"Sergio Luis Henao Montoya (Col) UAE Team Emirates",
235:"Juan Sebastian Molano Benavides (Col) UAE Team Emirates",
236:"Aleksandr Riabushenko (Blr) UAE Team Emirates",
237:"Maximiliano Ariel Richeze (Arg) UAE Team Emirates",
241:"Luca Wackermann (Ita) Vini Zabu' KTM",
242:"Marco Frapporti (Ita) Vini Zabu' KTM",
243:"James Mitri (NZl) Vini Zabu' KTM",
244:"Veljko Stojnic (Srb) Vini Zabu' KTM",
245:"Riccardo Stacchiotti (Ita) Vini Zabu' KTM",
246:"Giovanni Visconti (Ita) Vini Zabu' KTM",
247:"Edoardo Zardini (Ita) Vini Zabu' KTM",
"BORA - HANSGROHE":"Bora-Hansgrohe",
"ISRAEL START - UP NATION":"Israel Start-Up Nation",
"EF PRO CYCLING":"EF Pro Cycling",
"UAE TEAM EMIRATES":"UAE Team Emirates",
"AG2R LA MONDIALE":"AG2R la Mondiale",
"DECEUNINCK - QUICK - STEP":"Deceuninck-Quick-Step",
"CCC TEAM":"CCC Team",
"ALPECIN - FENIX":"Alpecin-Fenix",
"JUMBO - VISMA":"Team Jumbo-Visma",
"VINI ZABU' KTM":"Vini Zabu' KTM",
"GROUPAMA - FDJ":"Groupama-FDJ",
"TEAM SUNWEB":"Team Sunweb",
"BARDIANI CSF FAIZANE'":"Bardiani CSF Faizane'",
"MOVISTAR TEAM":"Movistar Team",
"NTT PRO CYCLING":"NTT Pro Cycling",
"ANDRONI GIOCATTOLI - SIDERMEC":"Androni Giocattoli-Sidermec",
"GAZPROM - RUSVELO":"Gazprom-Rusvelo",
"TOTAL DIRECT ENERGIE":"Total Direct Energie",
"BAHRAIN - MCLAREN":"Bahrain McLaren",
"TREK - SEGAFREDO":"Trek-Segafredo",
"MITCHELTON - SCOTT":"Mitchelton-Scott",
"ASTANA PRO TEAM":"Astana Pro Team",
"COFIDIS":"Cofidis",
"LOTTO SOUDAL":"Lotto Soudal",
"TEAM INEOS":"Ineos Grenadiers"}
frames = []
labels = []
def output_file(header_list, table_list):
    outfile = 'results.txt'
    with open(outfile, 'w') as file:
        for item in range(0, len(header_list)):
            try:
                file.write(f'\n{header_list[item]}\n')
                file.write(table_list[item].to_csv(header=False, index=False, encoding='UTF-8', sep=','))
            except:
                pass
    files.download(outfile)
# Take extraneous text out of the Bibs columns
def clean_points(df):    
    try:
        for item, index in df.iterrows():
            bibs = index['Bib']
            result = index['Result']
            if re.search('km', bibs):
                df.loc[item, 'Bib'] = ''
                df.loc[item, 'Result'] = ''
            elif re.search('More', bibs):
                df.drop(item, inplace=True)
            else:
                bibs = int(bibs)
                try:
                    df.loc[item, 'Bib'] = race_bibs[bibs]
                except:
                    pass

    except KeyError:
        print("Problem cleaning points")
        pass
    
    return df
    
def clean_times(df):

    try:
        for item, index in df.iterrows():
            time = index['Result']
            if re.search('^([0-9]{2})$', time):
                df.loc[item, 'Result'] = f'0:0:{time}'
            elif re.search('^[0-9]{2}:[0-9]{2}$', time):
                df.loc[item, 'Result'] = f'0:{time}'
            elif re.search('^[A-Z]{3}$', time):
                df.loc[item, 'Rank'] = time
                df.loc[item, 'Result'] = ''
    except KeyError:
        print("Problem cleaning times.")
        pass

    return df

def parse_tables(section, dtype):
    out_df = pd.DataFrame()

    for item in section:
        if dtype == 'Overall':
            title = item.find('h6').text
            title = f'{title} classification'
            print(title)
        else:
            title = item.find('h6').text
            print(title)
        # if dtype == 'Stage' and (title == 'Sprint' or title == 'Mountain'):
        #     # labels.append('')
        #     pass
        # else:
        #     labels.append(title)

        item = str(item).replace('style="display: none;"','')
        table = pd.read_html(item, index_col=None)[0]

        try:
            if 'Team' in title or title=='Team classification':
                out_df = table.drop(columns=[1,2,4,5], axis=1)
                out_df = out_df.rename(columns={0:'Rank',3:'Bib',6:'Result'})
                out_df['Result'] = out_df['Result'].str.replace('\' \'','').str.replace('"','').str.replace('\'',':').str.replace('h',':')
                out_df = clean_times(out_df)
                out_df['Bib'] = out_df['Bib'].map(race_bibs).fillna(out_df['Bib'])
                labels.append(title)
                frames.append(out_df)

            elif dtype == 'Stage' and 'Sprint' in title or dtype == 'Stage' and 'Mountain' in title:
                out_df = table.drop(columns=[1,3,4,5,6], axis=1)
                out_df = out_df.rename(columns={0:'Rank',2:'Bib',7:'Result'})
                clean_points(out_df)
                labels.append(f'{title}')
                frames.append(out_df)
            elif dtype == 'Overall' and 'Sprint' in title or dtype == 'Overall' and 'Mountain' in title:
                out_df = table.drop(columns=[1,3,4,5,6], axis=1)
                out_df = out_df.rename(columns={0:'Rank',2:'Bib',7:'Result'})
                out_df['Bib'] = out_df['Bib'].map(race_bibs).fillna(out_df['Bib'])
                labels.append(title)
                frames.append(out_df)
            else:
                out_df = table.drop(columns=[1,3,4,5,6], axis=1)
                out_df = out_df.rename(columns={0:'Rank',2:'Bib',7:'Result'})
                out_df['Result'] = out_df['Result'].str.replace('\' \'','').str.replace('"','').str.replace('\'',':').str.replace('h',':')
                out_df = clean_times(out_df)
                out_df['Bib'] = out_df['Bib'].map(race_bibs).fillna(out_df['Bib'])
                labels.append(title)  
                frames.append(out_df)                      
        except KeyError:
            print('Problem parsing tables')
            pass  

def scrape_tissot(url, dtype):
    
    page = requests.get(url)

    if page.status_code == 200:
        content = page.content
        soup = BeautifulSoup(content, "html5lib")
    try:
        section = soup.find_all('div', class_='ranking')

    except ElementDoesNotExist:
        print("Results not found - error in scrape_tissot")
    print(f'Scraped {dtype} - Processing')        
    parse_tables(section, dtype)

def start():
    stage_url = input("Enter the URL to Tissot results eg. https://www.tissottiming.com/2020/tia/en-us/default/Stage/1/Stage")
    base_url = stage_url.split('/')[0:9] #Race URL
    base_url = '/'.join(base_url)
    gc_url = f'{base_url}/overall#anchor'

    scrape_tissot(stage_url, 'Stage')
    scrape_tissot(gc_url, 'Overall')
    output_file(labels, frames)

# Press play to run
* Enter URL for stage 

In [51]:
frames = []
labels = []
start()


Enter the URL to Tissot results eg. https://www.tissottiming.com/2020/tia/en-us/default/Stage/1/Stagehttps://www.tissottiming.com/2020/tia/en-us/default/Stage/4/Stage
Scraped Stage - Processing
Stage
Sprint
Mountain
Best Young
Best Team
Scraped Overall - Processing
Individual classification
Sprint classification
Mountain classification
Youth classification
Team classification


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Debugging, ignore


In [38]:
url = 'https://www.tissottiming.com/2020/tia/en-us/default/Stage/2/Stage'
dtype = 'Stage'
page = requests.get(url)
frames = []
labels = []

if page.status_code == 200:
    content = page.content
    soup = BeautifulSoup(content, "html5lib")
try:
    section = soup.find_all('div', class_=dtype)

except ElementDoesNotExist:
    print("Results not found - error in scrape_tissot")

for item in section:
    title = item.find('h6').text
    item = str(item).replace('style="display: none;"','')
    table = pd.read_html(item, index_col=None)[0]
    
    frames.append(table)
    labels.append(title)
    out_df = pd.DataFrame()

    


In [47]:
# section
df = frames[1]
df = df.drop(columns=[1,3,4,5,6], axis=1)
df = df.rename(columns={0:'Rank',2:'Bib',7:'Result'})

# df = df.drop(columns=[1,3,4,5,6], axis=1)
# df = df.rename(columns={0:'Rank',2:'Bib',7:'Result'})

# df

# # outfile = 'output.csv'
# # with open(outfile, 'w') as file:
# #     for item in range(0, len(labels)):
# #         try:
# #             file.write(f'\n{labels[item]}\n')
# #             file.write(frames[item].to_csv(header=False, index=False, encoding='UTF-8'))
# #         except:
# #             pass
# # files.download(outfile)
# # outfile = 'test.csv'
# # with open(outfile, 'w') as file:
# #     file.write(frames[2].to_csv(header=False, index=False, encoding='Latin-1'))
# # files.download(outfile)

In [48]:
for item, index in df.iterrows():
    bibs = index['Bib']
    result = index['Result']
    if re.search('km', bibs):
        df.loc[item, 'Bib'] = ''
        df.loc[item, 'Result'] = ''
    elif re.search('More', bibs):
        df.drop(item, inplace=True)
    else:
        bibs = int(bibs)
        df.loc[item, 'Bib'] = race_bibs[bibs]

In [49]:
df

,Rank,Bib,Result
0,Canneto - km 137,,
1,1,Nicola Bagioli (Ita) Androni Giocattoli-Sidermec,5
2,2,Umberto Orsini (Ita) Bardiani CSF Faizane',3
3,3,Edoardo Zardini (Ita) Vini Zabu' KTM,2
4,4,Maciej Bodnar (Pol) Bora-Hansgrohe,1
5,Follonica - km 190,,
6,1,Pascal Ackermann (Ger) Bora-Hansgrohe,12
7,2,Fernando Gaviria Rendon (Col) UAE Team Emirates,10
8,3,Rick Zabel (Ger) Israel Start-Up Nation,8
9,4,Davide Ballerini (Ita) Deceuninck-Quickstep,7


In [81]:
dfs[1]

,Rank,Bib,Result
1,1,62,12
2,2,231,10
3,3,137,8
4,4,91,7
5,5,12,6
6,6,133,5
7,7,213,4
8,8,23,3
9,9,147,2
10,10,195,1


In [82]:
lbls[1]

'Follonica - km 190'

In [ ]:
        # dfs = []
        # lbls = []
        # res = [x.reset_index(drop=True) for x in np.split(df, np.where(df.applymap(lambda x: x == ''))[0]) if not x.empty]
        # for x in res:
        #     lbls.append(x.Rank.iloc[0])
        #     x = x.drop(x.index[0])
        #     dfs.append(x)